<a href="https://colab.research.google.com/github/harsh191/INTERNSHIP-PROJECTS/blob/master/Public_Transportation_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 98.0 MB/s eta 0:00:00


In [2]:
import pandas as pd
from pulp import *


In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
file_path='/content/drive/MyDrive/week 6 project/Internship_Major_Project.xlsx'
df=pd.read_excel(file_path,sheet_name='Cleaned data')
df.head()

,bus_id,route,Start_City,End_City,bus_type,depot,date,capacity,passengers,occupancy_rate,...,fare_per_passenger,revenue,Total_Fuel_Cost,fuel_consumed_liters,Profit,Total_Cost,month,day_of_week,Unnamed: 19,Unnamed: 20
0,AP0001,Kurnool-Hyderabad,Kurnool,Hyderabad,Volvo AC,Guntur,2024-11-26,49,32,65.77,...,908.75,29079.96,7282.9541,75.73,18797.0059,10282.9541,November,Tuesday,NaN,Average_Fuel_Price
1,AP0002,Anantapur-Bangalore,Anantapur,Bangalore,Sleeper,Tirupati,2024-10-03,36,14,41.42,...,770.66,10789.30,6880.9635,71.55,908.3365,9880.9635,October,Thursday,NaN,96.17
2,AP0003,Hyderabad-Vijayawada,Hyderabad,Vijayawada,Volvo AC,Vijayawada,2024-07-06,49,43,89.26,...,277.49,11932.08,4601.7345,47.85,4330.3455,7601.7345,July,Saturday,NaN,NaN
3,AP0004,Eluru-Hyderabad,Eluru,Hyderabad,Semi-Sleeper,Kurnool,2024-11-02,63,52,83.97,...,199.64,10381.38,6985.7888,72.64,395.5912,9985.7888,November,Saturday,NaN,NaN
4,AP0005,Kurnool-Hyderabad,Kurnool,Hyderabad,Sleeper,Hyderabad,2024-09-20,44,31,70.62,...,591.89,18348.44,7421.4389,77.17,7927.0011,10421.4389,September,Friday,NaN,Number_Distinct_Buses


In [5]:
clean_df=df.copy()
df.shape


(1000, 21)

In [6]:
# Clean and group by route, bus type, and depot
route_data = df.groupby(['route', 'bus_type', 'depot'], as_index=False).agg({
    'Profit': 'sum',
    'Total_Fuel_Cost': 'sum',
    'capacity': 'mean',
    'occupancy_rate': 'mean',
    'distance_km': 'mean'
})

print("Unique Routes:", len(route_data))
route_data.head()


Unique Routes: 497


,route,bus_type,depot,Profit,Total_Fuel_Cost,capacity,occupancy_rate,distance_km
0,Anantapur-Bangalore,Express,Hyderabad,-7467.1486,19193.6086,61.000000,28.375,406.200000
1,Anantapur-Bangalore,Express,Tirupati,702.6662,5495.1538,59.000000,47.800,308.340000
2,Anantapur-Bangalore,Express,Visakhapatnam,-13016.1693,25993.7893,56.333333,33.640,466.263333
3,Anantapur-Bangalore,Ordinary,Guntur,7800.1763,28813.4937,59.000000,78.100,342.425000
4,Anantapur-Bangalore,Ordinary,Hyderabad,13837.9247,12991.6053,59.000000,96.155,294.170000


In [7]:

df.head()

,bus_id,route,Start_City,End_City,bus_type,depot,date,capacity,passengers,occupancy_rate,...,fare_per_passenger,revenue,Total_Fuel_Cost,fuel_consumed_liters,Profit,Total_Cost,month,day_of_week,Unnamed: 19,Unnamed: 20
0,AP0001,Kurnool-Hyderabad,Kurnool,Hyderabad,Volvo AC,Guntur,2024-11-26,49,32,65.77,...,908.75,29079.96,7282.9541,75.73,18797.0059,10282.9541,November,Tuesday,NaN,Average_Fuel_Price
1,AP0002,Anantapur-Bangalore,Anantapur,Bangalore,Sleeper,Tirupati,2024-10-03,36,14,41.42,...,770.66,10789.30,6880.9635,71.55,908.3365,9880.9635,October,Thursday,NaN,96.17
2,AP0003,Hyderabad-Vijayawada,Hyderabad,Vijayawada,Volvo AC,Vijayawada,2024-07-06,49,43,89.26,...,277.49,11932.08,4601.7345,47.85,4330.3455,7601.7345,July,Saturday,NaN,NaN
3,AP0004,Eluru-Hyderabad,Eluru,Hyderabad,Semi-Sleeper,Kurnool,2024-11-02,63,52,83.97,...,199.64,10381.38,6985.7888,72.64,395.5912,9985.7888,November,Saturday,NaN,NaN
4,AP0005,Kurnool-Hyderabad,Kurnool,Hyderabad,Sleeper,Hyderabad,2024-09-20,44,31,70.62,...,591.89,18348.44,7421.4389,77.17,7927.0011,10421.4389,September,Friday,NaN,Number_Distinct_Buses


In [8]:
print(df.columns.tolist())


['bus_id', 'route', 'Start_City', 'End_City', 'bus_type', 'depot', 'date', 'capacity', 'passengers', 'occupancy_rate', 'distance_km', 'fare_per_passenger', 'revenue', 'Total_Fuel_Cost', 'fuel_consumed_liters', 'Profit', 'Total_Cost', 'month', 'day_of_week', 'Unnamed: 19', 'Unnamed: 20']


In [9]:
#  Data Cleaning and Preparation

# Clean and normalize all column names
df.columns = (
    df.columns
    .astype(str)
    .str.strip()
    .str.lower()
    .str.replace(r'\s+', '_', regex=True)
    .str.replace('\u00a0', '')
)


# Convert numeric columns
numeric_cols = ['profit', 'occupancy_rate', 'distance_km', 'fare_per_passenger',
                'total_fuel_cost', 'fuel_consumed_liters', 'capacity', 'passengers']
for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# occupancy_rate
if 'occupancy_rate' in df.columns:
    occ_max = df['occupancy_rate'].max(skipna=True)
    if pd.notna(occ_max) and occ_max > 1.5:
        # occupancy is in percent convert it to normal form
        df['occupancy_rate'] = df['occupancy_rate'] / 100.0
        print(" occupancy_rate detected in percent; converted to proportion (0-1).")

# Drop invalid profit rows
df = df.dropna(subset=['profit'])

# Unique variable identifier
df['var_name'] = df['bus_id'].astype(str) + "_" + df['route'].astype(str)


 occupancy_rate detected in percent; converted to proportion (0-1).


In [10]:
# Optimization 1
# Objective: Maximize Profit

# Model Setup

model = LpProblem("Bus_Route_Optimization", LpMaximize)

# Decision variables: number of trips per bus-route (0 to 3)
x = {
    r: LpVariable(r, lowBound=0, upBound=3, cat='Integer')
    for r in df['var_name']
}

# --- Objective Function: Adjusted Profit Maximization ---

# Penalize low-occupancy routes (<50%)
df['adjusted_profit'] = df.apply(
    lambda r: r['profit'] * 0.8 if r['occupancy_rate'] < 0.5 else r['profit'],
    axis=1
)

# Objective: maximize total adjusted profit
profit_map = df.set_index('var_name')['adjusted_profit'].to_dict()
model += lpSum(x[r] * profit_map[r] for r in x), "Total_Adjusted_Profit"

# --- Constraints ---

# 1. Depot limit: each depot can have at most trips in total
for depot in df['depot'].unique():
    depot_routes = df[df['depot'] == depot]['var_name']
    model += lpSum(x[r] for r in depot_routes) <= 180, f"DepotLimit_{depot}"

# 2. Long routes (>400 km) only allowed for Volvo buses
long_routes_mask = (df['distance_km'] > 400) & (df['bus_type'].str.lower() != 'volvo')
for r in df.loc[long_routes_mask, 'var_name']:
    model += x[r] == 0, f"LongRoute_Restriction_{r}"

# Optional (to include all buses): Each bus gets at least 1 route if desired
for bus in df['bus_id'].unique():
    bus_routes = df[df['bus_id'] == bus]['var_name']
    if len(bus_routes) > 0:
        model += lpSum(x[r] for r in bus_routes) >= 1, f"BusUse_{bus}"

# --- Solve the Model ---

print("\n Solving optimization model...")
model.solve(PULP_CBC_CMD(msg=True))

# --- Extract Results ---

results = []
for _, row in df.iterrows():
    trips = x[row['var_name']].value()
    if trips and trips > 0:
        results.append({
            'bus_id': row['bus_id'],
            'route': row['route'],
            'depot': row['depot'],
            'bus_type': row['bus_type'],
            'occupancy_rate': row['occupancy_rate'],
            'distance_km': row['distance_km'],
            'adjusted_profit': row['adjusted_profit'],
            'trips_selected': trips,
            'total_profit': row['adjusted_profit'] * trips
        })

optimized_routes1 = pd.DataFrame(results)
current_profit=df['profit'].sum()

# --- Display Results ---

if not optimized_routes1.empty:
    print("\n Optimized Routes:")
    print(optimized_routes1)
    print("\n Total Current Profit:", round(current_profit, 2))
    print("\n Total Optimized Profit:", round(optimized_routes1['total_profit'].sum(), 2))
    print(" Total Buses Used:", optimized_routes1['bus_id'].nunique())

else:
    print("\n No feasible solution found. Please check data or constraints.")



 Solving optimization model...

 Optimized Routes:
     bus_id                 route          depot      bus_type  \
0    AP0001     Kurnool-Hyderabad         Guntur      Volvo AC   
1    AP0002   Anantapur-Bangalore       Tirupati       Sleeper   
2    AP0003  Hyderabad-Vijayawada     Vijayawada      Volvo AC   
3    AP0004       Eluru-Hyderabad        Kurnool  Semi-Sleeper   
4    AP0006       Eluru-Hyderabad        Kurnool  Super Luxury   
..      ...                   ...            ...           ...   
826  AP0996   Kakinada-Vijayawada  Visakhapatnam       Sleeper   
827  AP0997   Anantapur-Bangalore     Vijayawada  Super Luxury   
828  AP0998    Hyderabad-Tirupati      Hyderabad       Express   
829  AP0999   Anantapur-Bangalore       Tirupati      Ordinary   
830  AP1000  Hyderabad-Vijayawada  Visakhapatnam      Ordinary   

     occupancy_rate  distance_km  adjusted_profit  trips_selected  \
0            0.6577       326.58      18797.00590             3.0   
1            0.41

In [11]:

#  Insights


if not optimized_routes1.empty:
    print("\n Insights:")
    print("=" * 40)

    #  Overall Summary
    total_profit = optimized_routes1['total_profit'].sum()
    avg_occupancy = optimized_routes1['occupancy_rate'].mean()
    avg_distance = optimized_routes1['distance_km'].mean()

    print(f" Total Optimized Profit: ₹{round(total_profit, 2)}")
    print(f" Average Occupancy Rate: {round(avg_occupancy * 100, 1)}%")
    print(f" Average Distance per Route: {round(avg_distance, 1)} km")

    #  Top 3 Routes by Profit
    top_routes = optimized_routes1.nlargest(3, 'total_profit')
    print("\n Top 3 Routes by Total Profit:")
    for _, r in top_routes.iterrows():
        print(f"  • Route {r['route']} ({r['bus_type']}) → ₹{round(r['total_profit'], 2)}")

    #  Depot Utilization
    depot_counts = optimized_routes1['depot'].value_counts()
    print("\n Depot Utilization (Trips Selected):")
    for depot, count in depot_counts.items():
        print(f"  • {depot}: {count} routes optimized")

    #  Long-Distance Route Insight
    long_routes = optimized_routes1[optimized_routes1['distance_km'] > 400]
    if not long_routes.empty:
        print(f" Long-distance routes assigned only to Volvo — count: {len(long_routes)} ")
    else:
        print(" No routes above 400 km optimized in this run.")
else:
    print("\n No optimization results available to analyze.")



 Insights:
 Total Optimized Profit: ₹11039758.9
 Average Occupancy Rate: 68.1%
 Average Distance per Route: 350.0 km

 Top 3 Routes by Total Profit:
  • Route Hyderabad-Vijayawada (Volvo AC) → ₹117965.6
  • Route Kadapa-Hyderabad (Volvo AC) → ₹117935.15
  • Route Chittoor-Bangalore (Volvo AC) → ₹114520.01

 Depot Utilization (Trips Selected):
  • Visakhapatnam: 137 routes optimized
  • Vijayawada: 136 routes optimized
  • Guntur: 120 routes optimized
  • Nellore: 118 routes optimized
  • Tirupati: 113 routes optimized
  • Kurnool: 109 routes optimized
  • Hyderabad: 98 routes optimized
 Long-distance routes assigned only to Volvo — count: 196 


In [12]:

# OPTIMIZATION 2


# --- Model Setup ---
model = LpProblem("Bus_Route_Optimization", LpMaximize)

# Allow more flexibility in trips (max 5 instead of 3)
x = {r: LpVariable(r, lowBound=0, upBound=5, cat='Integer') for r in df['var_name']}

# --- Objective Function ---
df['net_profit'] = df['profit'] - df['total_fuel_cost']

df['adjusted_net_profit'] = df.apply(
    lambda r: r['net_profit'] * 0.85 if r['occupancy_rate'] < 0.5 else r['net_profit'],
    axis=1
)

# Reduce penalty on fuel to encourage more buses
alpha = 1.0
beta = 0.25
df['weighted_obj'] = alpha * df['adjusted_net_profit'] - beta * df['fuel_consumed_liters']
objective_map = df.set_index('var_name')['weighted_obj'].to_dict()

model += lpSum(x[r] * objective_map[r] for r in x), "Weighted_NetProfit_Fuel"

# --- Constraints ---
# Depot limit – relaxed further (each depot can run up to 5× buses)
for depot in df['depot'].unique():
    depot_routes = df[df['depot'] == depot]['var_name']
    depot_bus_count = df[df['depot'] == depot]['bus_id'].nunique()
    model += lpSum(x[r] for r in depot_routes) <= depot_bus_count * 5, f"DepotLimit_{depot}"

# Long routes restriction
long_routes_mask = (df['distance_km'] > 400) & (df['bus_type'].str.lower() != 'volvo')
for r in df.loc[long_routes_mask, 'var_name']:
    model += x[r] == 0, f"LongRoute_{r}"

# Minimum occupancy relaxed (30% instead of 40%)
min_occupancy = 0.3
low_occ_mask = df['occupancy_rate'] < min_occupancy
for r in df.loc[low_occ_mask, 'var_name']:
    model += x[r] == 0, f"MinOccupancy_{r}"

# Encourage all buses to be used at least once (soft constraint)
bus_groups = df.groupby('bus_id')['var_name'].apply(list)
for bus_id, routes in bus_groups.items():
    model += lpSum(x[r] for r in routes) <= 10, f"BusTripLimit_{bus_id}"
    model += lpSum(x[r] for r in routes) >= 0, f"BusUsageLower_{bus_id}"


# --- Solve ---
print("\n Solving optimization model...")
model.solve(PULP_CBC_CMD(msg=True, timeLimit=300))

# --- Extract Results ---
results = []
for _, row in df.iterrows():
    trips = x[row['var_name']].value()
    if trips and trips > 0:
        results.append({
            'bus_id': row['bus_id'],
            'route': row['route'],
            'depot': row['depot'],
            'bus_type': row['bus_type'],
            'occupancy_rate': row['occupancy_rate'],
            'distance_km': row['distance_km'],
            'adjusted_net_profit': row['adjusted_net_profit'],
            'fuel_consumed_liters': row['fuel_consumed_liters'],
            'trips_selected': trips,
            'total_net_profit': row['adjusted_net_profit'] * trips,
            'total_fuel_used': row['fuel_consumed_liters'] * trips
        })

optimized_routes2 = pd.DataFrame(results)
current_profit=df['profit'].sum()

# --- Display ---
print("\n Optimized Routes :")
print(optimized_routes2.head(15))
print("\n Total Current Profit:", round(current_profit, 2))
print("\n Total Optimized Net Profit:", round(optimized_routes2['total_net_profit'].sum(), 2))
print(" Total Fuel Used:", round(optimized_routes2['total_fuel_used'].sum(), 2))
print(" Total Buses Used:", optimized_routes2['bus_id'].nunique(), "/", df['bus_id'].nunique())

# --- Insights ---
if not optimized_routes2.empty:
    print("\n Insights:")
    print("=" * 45)
    total_profit = optimized_routes2['total_net_profit'].sum()
    total_fuel = optimized_routes2['total_fuel_used'].sum()
    avg_occ = optimized_routes2['occupancy_rate'].mean()
    avg_distance = optimized_routes2['distance_km'].mean()

    print(f" Total Net Profit: ₹{round(total_profit, 2)}")
    print(f" Total Fuel Used: {round(total_fuel, 1)} liters")
    print(f" Average Occupancy Rate: {round(avg_occ * 100, 1)}%")
    print(f" Average Distance: {round(avg_distance, 1)} km")

    top3 = optimized_routes2.nlargest(3, 'total_net_profit')
    print("\n Top 3 Most Profitable Routes:")
    for _, r in top3.iterrows():
        print(f"  • {r['route']} ({r['bus_type']}) → ₹{round(r['total_net_profit'], 2)}")

    depot_counts = optimized_routes2['depot'].value_counts()
    print("\n Depot Utilization:")
    for depot, count in depot_counts.items():
        print(f"  • {depot}: {count} optimized routes")

    efficient = optimized_routes2.assign(
        profit_per_liter=lambda d: d['total_net_profit'] / d['total_fuel_used']
    ).sort_values('profit_per_liter', ascending=False).iloc[0]
    print(f"\n Most Fuel-Efficient Route: {efficient['route']} ({efficient['bus_type']})")
else:
    print("\n No optimization results found.")



 Solving optimization model...

 Optimized Routes :
    bus_id                     route       depot      bus_type  \
0   AP0001         Kurnool-Hyderabad      Guntur      Volvo AC   
1   AP0006           Eluru-Hyderabad     Kurnool  Super Luxury   
2   AP0007          Guntur-Hyderabad     Nellore       Sleeper   
3   AP0013      Hyderabad-Vijayawada  Vijayawada       Sleeper   
4   AP0015          Kadapa-Hyderabad      Guntur  Super Luxury   
5   AP0016        Chittoor-Bangalore    Tirupati      Volvo AC   
6   AP0018      Hyderabad-Vijayawada  Vijayawada  Super Luxury   
7   AP0022      Hyderabad-Vijayawada      Guntur  Semi-Sleeper   
8   AP0024  Vijayawada-Visakhapatnam   Hyderabad      Volvo AC   
9   AP0031       Anantapur-Bangalore  Vijayawada  Super Luxury   
10  AP0033       Vijayawada-Tirupati   Hyderabad       Sleeper   
11  AP0036          Guntur-Hyderabad  Vijayawada       Sleeper   
12  AP0037  Vijayawada-Visakhapatnam    Tirupati       Sleeper   
13  AP0040       Vijaya

In [13]:
# Optimization 3

#  Model Setup

model = LpProblem("Bus_Route_Optimization", LpMaximize)
x = {r: LpVariable(r, lowBound=0, upBound=3, cat='Integer') for r in df['var_name']}



# Net profit
df['net_profit'] = df['profit'] - df['total_fuel_cost']

# Adjust net profit for low occupancy
df['adjusted_net_profit'] = df.apply(
    lambda r: r['net_profit'] * 0.8 if r['occupancy_rate'] < 0.5 else r['net_profit'], axis=1
)

# Seat efficiency bonus
df['seat_efficiency'] = df['occupancy_rate'] * df['capacity']

# Weighted objective: profit, fuel minimization, seat efficiency
alpha = 1.0  # net profit weight
beta = 0.5   # fuel minimization weight
gamma = 0.2  # seat efficiency weight

df['weighted_obj'] = (alpha*df['adjusted_net_profit']
                      - beta*df['fuel_consumed_liters']
                      + gamma*df['seat_efficiency'])


objective_map = df.set_index('var_name')['weighted_obj'].to_dict()
model += lpSum(x[r] * objective_map[r] for r in x), "Weighted_MultiObjective"


# Constraints

# Depot limit:
for depot in df['depot'].unique():
    depot_routes = df[df['depot']==depot]['var_name']
    model += lpSum(x[r] for r in depot_routes) <= 180, f"DepotLimit_{depot}"

# Long routes (>400 km) only for Volvo buses
long_routes_mask = (df['distance_km'] > 400) & (df['bus_type'].str.lower() != 'volvo')
for r in df.loc[long_routes_mask, 'var_name']:
    model += x[r] == 0, f"LongRoute_{r}"

# Minimum occupancy (40%)
min_occupancy = 0.4
low_occ_mask = df['occupancy_rate'] < min_occupancy
for r in df.loc[low_occ_mask, 'var_name']:
    model += x[r] == 0, f"MinOccupancy_{r}"

# Max distance per bus per day
max_distance_per_bus = 3000
for _, row in df.iterrows():
    model += x[row['var_name']] * row['distance_km'] <= max_distance_per_bus, f"MaxDistance_{row['var_name']}"



#  Solve

print("\n Solving optimization model...")
model.solve(PULP_CBC_CMD(msg=True))


#  Extract Results

results = []
for _, row in df.iterrows():
    trips = x[row['var_name']].value()
    if trips and trips > 0:
        results.append({
            'bus_id': row['bus_id'],
            'route': row['route'],
            'depot': row['depot'],
            'bus_type': row['bus_type'],
            'occupancy_rate': row['occupancy_rate'],
            'distance_km': row['distance_km'],
            'adjusted_net_profit': row['adjusted_net_profit'],
            'trips_selected': trips,
            'total_net_profit': row['adjusted_net_profit'] * trips,
            'fuel_consumed_liters': row['fuel_consumed_liters'],
            'total_fuel_used': row['fuel_consumed_liters'] * trips,
            'seat_efficiency': row['seat_efficiency']
        })

optimized_routes3 = pd.DataFrame(results)
current_profit=df['profit'].sum()


# Display Results

display_columns = [
    'bus_id', 'route', 'depot', 'bus_type', 'occupancy_rate', 'distance_km',
    'adjusted_net_profit', 'trips_selected', 'total_net_profit',
    'fuel_consumed_liters', 'total_fuel_used', 'seat_efficiency'
]
optimized_routes_display = optimized_routes3[display_columns]

print("\nOptimized Routes (Net Profit First):")
print(optimized_routes_display)
print("\n Total Current Profit:", round(current_profit, 2))
print("\n Total Optimized Net Profit:", round(optimized_routes_display['total_net_profit'].sum(), 2))
print(" Total Fuel Consumed:", round(optimized_routes_display['total_fuel_used'].sum(), 2))


# Insights


if not optimized_routes3.empty:
    print("\n Insights Summary:")
    print("=" * 50)

    # Key Metrics
    total_profit = optimized_routes3['total_net_profit'].sum()
    total_fuel = optimized_routes3['total_fuel_used'].sum()
    avg_occ = optimized_routes3['occupancy_rate'].mean()
    avg_distance = optimized_routes3['distance_km'].mean()
    avg_seat_eff = optimized_routes3['seat_efficiency'].mean()

    print(f"  Total Net Profit: ₹{round(total_profit, 2)}")
    print(f"  Total Fuel Used: {round(total_fuel, 1)} liters")
    print(f"  Average Seat Efficiency: {round(avg_seat_eff, 2)}")
    print(f"  Average Occupancy Rate: {round(avg_occ * 100, 1)}%")
    print(f"  Average Route Distance: {round(avg_distance, 1)} km")


    # Top 3 Profitable Routes
    top3 = optimized_routes3.nlargest(3, 'total_net_profit')
    print("\n  Top 3 Most Profitable Routes:")
    for _, r in top3.iterrows():
        print(f"  • {r['route']} ({r['bus_type']}) → ₹{round(r['total_net_profit'], 2)}")

    # Depot Utilization
    depot_counts = optimized_routes3['depot'].value_counts()
    print("\n  Depot Utilization Summary:")
    for depot, count in depot_counts.items():
        print(f"  • {depot}: {count} optimized routes")


    # Most Fuel-Efficient Route
    fuel_eff = optimized_routes3.copy()
    fuel_eff['profit_per_litre'] = fuel_eff['total_net_profit'] / fuel_eff['total_fuel_used']
    efficient = fuel_eff.nlargest(1, 'profit_per_litre').iloc[0]

    print(f"\n  Most Fuel-Efficient Route:")
    print(f"  • {efficient['route']} ({efficient['bus_type']}) ")


    # Highest Seat Efficiency
    top_seat = optimized_routes3.nlargest(1, 'seat_efficiency').iloc[0]
    print(f"\n  Highest Seat Efficiency Route:")
    print(f"  • {top_seat['route']} ({top_seat['bus_type']})")


else:
    print("\n  No optimization results found. Please check model constraints or weights.")




 Solving optimization model...

Optimized Routes (Net Profit First):
     bus_id                 route          depot      bus_type  \
0    AP0001     Kurnool-Hyderabad         Guntur      Volvo AC   
1    AP0006       Eluru-Hyderabad        Kurnool  Super Luxury   
2    AP0007      Guntur-Hyderabad        Nellore       Sleeper   
3    AP0013  Hyderabad-Vijayawada     Vijayawada       Sleeper   
4    AP0015      Kadapa-Hyderabad         Guntur  Super Luxury   
..      ...                   ...            ...           ...   
287  AP0969     Kurnool-Hyderabad  Visakhapatnam       Sleeper   
288  AP0970  Hyderabad-Vijayawada  Visakhapatnam      Volvo AC   
289  AP0977      Guntur-Hyderabad        Nellore       Sleeper   
290  AP0979      Kadapa-Hyderabad        Nellore       Express   
291  AP0992      Guntur-Hyderabad     Vijayawada  Super Luxury   

     occupancy_rate  distance_km  adjusted_net_profit  trips_selected  \
0            0.6577       326.58           11514.0518           

In [14]:
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 4.5 MB/s eta 0:00:00


In [15]:


# Saving All Optimization In Single Excel File

# File name
output_file = "all_optimized_routes.xlsx"

# Save multiple DataFrames to different sheets
with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
    optimized_routes1.to_excel(writer, sheet_name='Optimized_Routes1', index=False)
    optimized_routes2.to_excel(writer, sheet_name='Optimized_Routes2', index=False)
    optimized_routes3.to_excel(writer, sheet_name='Optimized_Routes3', index=False)

print(f"\n All optimized results saved to '{output_file}' with multiple sheets.")



 All optimized results saved to 'all_optimized_routes.xlsx' with multiple sheets.
